# VSSo + SOSA synthetic data generator
This notebook generates example data of a `Vehicle` using the [Vehicle Signal Specification Ontology (VSSo)](https://github.com/w3c/vssohttps://github.com/w3c/vsso).
According to the current status of VSSo (as of July 2021), a `Vehicle` has properties that can be static or dynamic.
* An `StaticVehicleProperty` (i.e., attribute) is a characteristic of the `Vehicle` whose value does not change often (e.g., `VehicleIdentificationBrand`).
* In contrast, a `DynamicVehicleProperty` (i.e., signal) is a characteristic of the `Vehicle` whose value changes frequently, resulting in time-series data (aka., data stream).

The synthetic data that this notebook creates is of the type time-series (i.e., sequences of values over time).
The resulting data showcases the functionality of the **VSSo core** model together with the [Sensor, Observation, Sample, and Actuator (SOSA) ontology](https://www.w3.org/TR/vocab-ssn/https://www.w3.org/TR/vocab-ssn/][https://www.w3.org/TR/vocab-ssn/https://www.w3.org/TR/vocab-ssn/).

> An example of the use of the **VSSo core** model (i.e., not time-series data) is available [here](https://github.com/danielwilms/vsso-demo/tree/main/examples/vsso-core).

## Instructions
Almost all values are generated randomly within a range that is appropiate to each property.
If other values or more properties are desired, simply modify the corresponding line.

You can run this notebook using [Jupyter](https://jupyter.org/install).
Make sure that you are using a kernel that has already intalled the `environment.yml` file available in the working directory.
If you use `conda`, do the following in the working directory:
```zsh
conda env create --name vsso-demo --file environment.yml
conda activate vsso-demo
jupyter lab
```

To create a new `vsso+sosa_data_example.ttl`, just run all cells sequentially.
> In a Jupyter noteboook do: `Kernel`>`Restart Kernel and Run All Cells...`

### Dependencies

In [1]:
import random
import string
from rdflib import URIRef, Literal, Namespace, RDF, Graph, XSD

### RDF graph configuration

In [2]:
num_vehicles = 1  # how many vehicles you want to simulate?
num_observations = 10  # how many sequential observations you want for the dynamic properties?

In [3]:
rdf_graph = Graph()  # create an empty RDF graph

vsso = Namespace("https://github.com/danielwilms/vsso-demo/")  # default root URI of the VSSo ontology
indv = Namespace("https://individualsnamespace.test/vsso-demo/")  # root URI used for the individuals
sosa = Namespace("http://www.w3.org/ns/sosa/")  # SOSA ontology

In this example, we create data related to the following battery aspects of an electric vehicle:
* Static
  * `VehicleIdentificationVIN`
  * `BatteryGrossCapacity`
* Dynamic
  * `


In [4]:
def add_static_properties(graph, vehicle_identification_number, vehicle_indvividual, special_properties=None):
    '''Adds the static properties to the given RDF graph.'''
    
    static_vehicle_properties = [
        {'name':'VehicleIdentificationVIN', 'value':str(vehicle_identification_number)},
        {'name':'BatteryGrossCapacity', 'value':str(random.choice(list(range(40,50))))},
        # Add here other static vehicle properties, if needed
    ]
    
    for i in static_vehicle_properties:        
        name = i['name']
        individual_uri = getattr(indv, f'Vehicle-{vehicle_identification_number}-{name}')
        class_name = name.split('-')[0]
        class_uri = getattr(vsso, f'{class_name}')

        graph.add((vehicle_indvividual, vsso.hasStaticVehicleProperty, individual_uri))
        graph.add((individual_uri, RDF.type, class_uri))
        graph.add((individual_uri, vsso.holdsState, Literal(i['value'])))   

In [5]:
def add_dynamic_properties(graph, vehicle_identification_number, vehicle_indvividual, special_properties=None):
    '''Adds the dynamic properties to the given RDF graph.'''
    
    dynamic_vehicle_properties = [
        {'name':'StateOfChargeCurrent', 'value':random.randint(50, 100)},
        # Add here other dynamic vehicle properties, if needed
    ]
        
    for i in dynamic_vehicle_properties:
        current_value = i['value']
        name = i['name']
        individual_uri = getattr(indv, f'Vehicle-{vehicle_identification_number}-{name}')
        class_name = name.split('-')[0]
        class_uri = getattr(vsso, f'{class_name}')
        
        sensor_uri = getattr(indv, f'Vehicle-{vehicle_identification_number}-{name}-Sensor')
        
        graph.add((vehicle_indvividual, vsso.hasDynamicVehicleProperty, individual_uri))  # Vehicle hasDVP DVP
        graph.add((individual_uri, RDF.type, class_uri))  # DVP is a vsso class

        graph.add((sensor_uri, RDF.type, sosa.Sensor))

        for t in range(num_observations):
            current_value = int(current_value * 0.95)  # simulate that the value is decreasing over time

            observation_uri = getattr(indv, f'Vehicle-{vehicle_identification_number}-{name}-Sensor-Observation-{t}')
            
            # Make sure that the holdsState has only the latest value
            graph.remove((individual_uri, vsso.holdsState, None))  
            graph.add((individual_uri, vsso.holdsState, Literal(current_value, datatype=XSD.int)))  # DVP holdsState Literal
            
            # Add the current Observation
            graph.add((observation_uri, RDF.type, sosa.Observation))
            graph.add((sensor_uri, sosa.observes, individual_uri))  # Sensor observes DynamicVehicleProperty
            graph.add((sensor_uri, sosa.madeObservation, observation_uri))  # Sensor madeObservation Observation
            graph.add((observation_uri, sosa.hasSimpleResult, Literal(current_value, datatype=XSD.int)))  # Observation hasSimpleResult "value"^^xsd:float
            graph.add((observation_uri, sosa.resultTime, Literal(t, datatype=XSD.dateTime)))  # Observation resultTime "2017-06-06T12:36:12Z"^^xsd:dateTime

In [6]:
# Loop over all the desired vehicles
for vehicle_identification_number in range(num_vehicles):
    # Create Vehicle individual using the VIN
    vehicle_indvividual = getattr(indv, f'Vehicle-{vehicle_identification_number}')
    
    rdf_graph.add((vehicle_indvividual, RDF.type, vsso.Vehicle))
    
    add_static_properties(rdf_graph, vehicle_identification_number, vehicle_indvividual)
    add_dynamic_properties(rdf_graph, vehicle_identification_number, vehicle_indvividual)

In [15]:
# Bind the prefixes for the abbreviated serialization
rdf_graph.bind("vsso", "https://github.com/danielwilms/vsso-demo/")
rdf_graph.bind("indv", URIRef("https://individualsnamespace.test/vsso-demo/"))
rdf_graph.bind("sosa", "http://www.w3.org/ns/sosa/")
rdf_graph.serialize("vsso+sosa_data_example.ttl", format='ttl')

In [16]:
print(rdf_graph.serialize(format='ttl'))

@prefix indv: <https://individualsnamespace.test/vsso-demo/> .
@prefix sosa: <http://www.w3.org/ns/sosa/> .
@prefix vsso: <https://github.com/danielwilms/vsso-demo/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

indv:Vehicle-0 a vsso:Vehicle ;
    vsso:hasDynamicVehicleProperty indv:Vehicle-0-StateOfChargeCurrent ;
    vsso:hasStaticVehicleProperty indv:Vehicle-0-BatteryGrossCapacity,
        indv:Vehicle-0-VehicleIdentificationVIN .

indv:Vehicle-0-StateOfChargeCurrent-Sensor a sosa:Sensor ;
    sosa:madeObservation indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-0,
        indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-1,
        indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-2,
        indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-3,
        indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-4,
        indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-5,
        indv:Vehicle-0-StateOfChargeCurrent-Sensor-Observation-6,
        indv:Ve

Notice that the relationship `vsso:holdsState` is intended to hold the latest state (i.e., value) of the vehicle property.
Therefore, its value will always correspond to the last `sosa:observation`.